In [1]:
import numpy as np
from face_geometry import cpp_compare, log, project_xy,\
                          change_handedness, estimate_scale,\
                          move_and_rescale_z, unproject_xy,\
                          canonical_metric_landmarks, landmark_weights, solve_weighted_orthogonal_problem,\
                          get_metric_landmarks,\
                          DEBUG, PCF

In [2]:
# save a file with your face from your camera with as frame.jpeg, then run jpeg_to_landmark.py
# thereafter run this cell, then the cpp script (main.cpp), and then the next cell
landmarks = np.load('landmarks.npy')
np.save('canonical_metric_landmarks.npy', canonical_metric_landmarks)
np.save('landmark_weights.npy', np.stack((landmark_weights, landmark_weights)))

In [3]:
pcf = PCF()

# this line DEBUG.get_debug() is False
DEBUG.toggle()

cpp_compare("canonical_metric_landmarks", canonical_metric_landmarks)
cpp_compare("landmarks", landmarks)

DEBUG.toggle()

screen_landmarks = landmarks.copy()

screen_landmarks = project_xy(screen_landmarks, pcf)
depth_offset = np.mean(screen_landmarks[2, :])

intermediate_landmarks = screen_landmarks.copy()
intermediate_landmarks = change_handedness(intermediate_landmarks)
first_iteration_scale = estimate_scale(intermediate_landmarks)
print("first_iteration_scale:", first_iteration_scale)

DEBUG.toggle()

intermediate_landmarks = screen_landmarks.copy()
intermediate_landmarks = move_and_rescale_z(pcf, depth_offset, first_iteration_scale, intermediate_landmarks)
intermediate_landmarks = unproject_xy(pcf, intermediate_landmarks)
intermediate_landmarks = change_handedness(intermediate_landmarks)
second_iteration_scale = estimate_scale(intermediate_landmarks)
print("second_iteration_scale:", second_iteration_scale)

metric_landmarks = screen_landmarks.copy()
total_scale = first_iteration_scale * second_iteration_scale
metric_landmarks = move_and_rescale_z(pcf, depth_offset, total_scale, metric_landmarks)
metric_landmarks = unproject_xy(pcf, metric_landmarks)
metric_landmarks = change_handedness(metric_landmarks)

pose_transform_mat = solve_weighted_orthogonal_problem(canonical_metric_landmarks, metric_landmarks, landmark_weights)

cpp_compare("pose_transform_mat", pose_transform_mat)

inv_pose_transform_mat = np.linalg.inv(pose_transform_mat)
cpp_compare("inv_pose_transform_mat", inv_pose_transform_mat)

inv_pose_rotation = inv_pose_transform_mat[:3, :3]
cpp_compare("inv_pose_rotation", inv_pose_rotation)

inv_pose_translation = inv_pose_transform_mat[:3, 3]
log("inv_pose_translation", inv_pose_translation)

metric_landmarks = inv_pose_rotation @ metric_landmarks + inv_pose_translation[:, None]

cpp_compare("metric_landmarks", metric_landmarks)

DEBUG.toggle()

metric_landmarks_2, _ = get_metric_landmarks(landmarks.copy(), pcf)

print("Check get_metric_landmarks function", np.sum(np.abs(metric_landmarks_2-metric_landmarks)**2))

canonical_metric_landmarks: 1.490073712786996e-11

landmarks: 0.0

first_iteration_scale: 0.020541233497050223
sources: 1.490073712786996e-11

targets: 433.8015801495133

weighted_sources: 8.256299453950529e-14

weighted_targets: 0.9832847159127093

total_weight logged: 1.0550000043585919

source_center_of_mass logged: [ 1.05234409e-16 -2.62520349e-01  3.74073921e+00]

centered_weighted_sources: 7.335573533966446e-13

design_matrix: 0.1816263033073122

design_matrix_norm logged: 21.802313842814225

postrotation: 4.00000038352286

prerotation: 4.000000730636378

rotation: 2.4840596720560027e-13

scale logged: 1.019935273598653

pointwise_diffs: 1.1283540472641884

weighted_pointwise_diffs: 0.07012732692605365

translation logged: [ -1.57876687   2.51696509 -52.74754359]

transform_mat: 0.00119223484434846

second_iteration_scale: 1.0199352735986524
sources: 1.490073712786996e-11

targets: 3.5416487308627116e-06

weighted_sources: 8.256299453950529e-14

weighted_targets: 7.92541611920077